In [1]:
%config IPCompleter.greedy=True
import pandas as pd
from datetime import datetime

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database



import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Define parameters:

In [2]:
startTime = '2019-08-01 00:00'
resolution = 15 #min
sensorCount = 10
datapointCount = 96


## Create Table with data

In [3]:
#Timestamp column
timestamps=pd.date_range(pd.to_datetime(startTime), periods=datapointCount, freq=str(resolution)+'min')
timestamps=np.repeat(timestamps.values,sensorCount,axis=0)

#Sensor ID column
sensorIds=["S"+str(x) for x in range(sensorCount)]
sensorIdColumns=sensorIds*datapointCount

#sensor data column - float values between 0 and 100
np.random.seed(0)
randomData=np.random.rand(datapointCount*sensorCount,1)*100
    
data={'sensorID':sensorIdColumns,'time':timestamps, 'value':randomData.flatten()}
dataTable=pd.DataFrame(data)

dataTable.head(10)

,sensorID,time,value
0,S0,2019-08-01,54.881350
1,S1,2019-08-01,71.518937
2,S2,2019-08-01,60.276338
3,S3,2019-08-01,54.488318
4,S4,2019-08-01,42.365480
5,S5,2019-08-01,64.589411
6,S6,2019-08-01,43.758721
7,S7,2019-08-01,89.177300
8,S8,2019-08-01,96.366276
9,S9,2019-08-01,38.344152


## Write to Database

In [4]:
user='postgres'
passw='docker'
host='localhost'
port='5432'
db='Sensors'

Connect to database

In [6]:
print("Write data to database ...")
engine=create_engine('postgresql://'+user+':'+passw+'@'+host+':'+port+'/'+db)

if not database_exists(engine.url):
    create_database(engine.url)

dataTable.to_sql('SensorData',engine,if_exists='replace')
print("done!")

Write data to database ...
done!


## Create individuals for ontology in RDF

In [7]:
resolution
sensorCount 
datapointCount
outputFileName='outputRDF_Res'+str(resolution)+'_Sens'+ str(sensorCount)+'_DP'+str(datapointCount)+'.ttl'

from rdflib import BNode, Graph, URIRef, Literal, Namespace, RDF
sosaNsString='http://www.w3.org/ns/sosa/'
sosa = Namespace(sosaNsString)

#g=Graph(store='Sleepycat')
#g.open('GeneratedTripes',create=True)
g=Graph()

    

print("Sensor: " + str(sensorIds))
#add every sensor
for s in sensorIds:    
    g.add((URIRef(sosaNsString + s), RDF.type, sosa.Sensor)) 

#add observation for every row in table
for index, row in dataTable.iterrows():
    #print(index,row["sensorID"],row["time"],row["value"])
    #Add observation
    g.add((URIRef(sosaNsString + "Observation_" + str(index)), RDF.type, sosa.Observation))
    
    # add time and result to observation
    g.add((URIRef(sosaNsString + "Observation_" + str(index)), sosa.resultTime, Literal(row['time'])))
    g.add((URIRef(sosaNsString + "Observation_" + str(index)), sosa.hasSimpleResult, Literal(row['value'])))
    
    # observation madeBySensor sensor
    g.add((URIRef(sosaNsString + "Observation_" + str(index)), sosa.madeBySensor, URIRef(sosaNsString + row["sensorID"])))
    
    # sensor made Observation observation
    g.add((URIRef(sosaNsString + row["sensorID"]), sosa.madeObservation ,sosaNsString + URIRef("Observation_" + str(index))))
    


#print(g.serialize(format='turtle'))

g.serialize(destination=outputFileName, format='turtle')
g.close()

Sensor: ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']


## Create Mapping File

In [5]:
outputFileName='Mapping_Res'+str(resolution)+'_Sens'+ str(sensorCount)+'_DP'+str(datapointCount)+'.ttl'

from rdflib import BNode, Graph, URIRef, Literal, Namespace, RDF

sosaNsString='http://www.w3.org/ns/sosa/'
sosa = Namespace(sosaNsString)
mappingString ='http://sic.auto.tuwien.ac.at/mappings#'
mapping =Namespace(mappingString)

#g=Graph(store='Sleepycat')
#g.open('GeneratedTripes',create=True)
g=Graph()

g.add((URIRef(mappingString+"Connection1"), mapping.url, Literal("localhost:5432")))
g.add((URIRef(mappingString+"Connection1"), mapping.db, Literal("Sensors")))
g.add((URIRef(mappingString+"Connection1"), mapping.user, Literal("postgres")))     
g.add((URIRef(mappingString+"Connection1"), mapping.passw, Literal("docker")))
      
for s in sensorIds:    
    sqlString="Select \"time\", \"value\" FROM \"SensorData\" WHERE (\"sensorID\" = '"+ s +"') ORDER BY \"time\" DESC "
    g.add((URIRef(sosaNsString + s), mapping.hasMapping, URIRef(mappingString + "Mapping"+s)))
    g.add((URIRef(mappingString + "Mapping"+s), mapping.hasDBConnection, URIRef(mappingString+"Connection1")))
    g.add((URIRef(mappingString + "Mapping"+s), mapping.hasSQLString, Literal(sqlString)))

g.serialize(destination=outputFileName, format='turtle')
g.close()

## Test Jena Fuseki

In [14]:
import time
import requests
import json
from datetime import datetime


writeFile="results.txt"

filePath="./SPARQL Queries/"
#queries=["Q1_obda","Q2_obda","Q3_obda","Q4_obda","Q5_obda"]
queries=["Q1","Q2","Q3","Q4","Q5"]

logTime=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
logString= "\n"+logTime + " - Res: " +str(resolution) + ", Sensors: "+ str(sensorCount) +", points/sensor: "+ str(datapointCount)+"\n"
f=open(writeFile,"a+")
f.write(logString)
f.write("------------------\n")
f.close()

for q in queries:
    currentFilePath=filePath+q+".txt"
    f= open(currentFilePath,"r")
    testQuery= f.read()
    f.close()
     
    start=time.time()
    response = requests.post('http://127.0.0.1:3030/ds', data={'query': testQuery})
    end=time.time()
    dtime=end-start
    print(dtime)
    data = response.json()
    dataCount= len(data['results']['bindings'])
    logTime=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    logString=logTime + " - "+q+" - "+str(dataCount) +" - " +str(len(response.content))+ " Byte" 
    
    f=open(writeFile,"a+")
   
    if( dataCount > 0):
        f.write(logString  + " - Execution Time " +str(dtime))
    else:
        f.write(logString +" - failed Response")
                
    f.write("\n")
    f.close() 
    
   
   

2.575029134750366
1.5109972953796387
2.448993682861328
0.9150176048278809
2.2129952907562256


## Get File Size

In [6]:
import os

filename= "./apache-jena-fuseki-3.12.0/run/databases/ds"

#os.path.getsize(./apache-jena-fuseki-3.12.0/fuseki-server-startWithOBDA.bat')


def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

print(get_size(filename))

0


## Start Fuseki


In [2]:
import os
import subprocess
import time
import sys

def runMyFile():
   # try:
    os.startfile('C:/Users/guser/Documents/CustomPropertyODBA/apache-jena-fuseki-3.12.0/fuseki-server-startWithOBDA.bat')

cmd="start C:/Users/guser/Documents/CustomPropertyODBA/apache-jena-fuseki-3.12.0/fuseki-server-startWithOBDA.bat"
wd="C:/Users/guser/Documents/CustomPropertyODBA/apache-jena-fuseki-3.12.0"
p = subprocess.Popen(cmd, shell=True, stdout = subprocess.PIPE, cwd=wd)


print("waiting 5 seconds...")
time.sleep(5)
print("kill process")
p.terminate()
print(p.returncode) # is 0 if success
#subprocess.call([r'C:/Users/guser/Documents/CustomPropertyODBA/apache-jena-fuseki-3.12.0/fuseki-server-startWithOBDA.bat'])

waiting 5 seconds...
kill process
0


In [2]:
import os

directory="C:/Users/guser/Documents/CustomPropertyODBA/apache-jena-fuseki-3.12.0/"
cmd="start \"Fuseki\" fuseki-server-startWithOBDA.bat"

wd=os.getcwd()
os.chdir(directory)
os.system(cmd)


0

## Delete and create dataset in fuseki

In [38]:
#delete database
response = requests.delete('http://127.0.0.1:3030/$/datasets/ds2')
#/$/datasets/*{name}*
print(response)

<Response [404]>


In [41]:
payload = {'dbType': 'tdb', 'dbName': 'ds2'}
headers = {'Content-Type': 'text/turtle;charset=utf-8'}
r = requests.post('http://localhost:3030/$/datasets', headers=headers, params=payload)
print(r)


<Response [200]>


In [42]:
data = open('sosa.ttl').read()
r = requests.post('http://localhost:3030/ds2/data', data=data, headers=headers,)
print(r)


<Response [200]>


In [16]:
response

<Response [405]>